In [1]:
import random
import collections
import numpy as np
from copy import deepcopy
import tkinter as tk

In [6]:
def score(acode,aguess):
    
    CodeColorCount=collections.Counter(acode)
    GuessColorCount=collections.Counter(aguess)
    blanc=sum(min(CodeColorCount[k],GuessColorCount[k]) for k in CodeColorCount)
    noir=sum(a==b for a,b in zip(acode,aguess))
    blanc-=noir
    
    return noir,blanc

In [7]:
def running(acode):
    guess=input("->")
    noir, blanc = score(acode,guess)
    print('noir: {}, blanc: {}'.format(noir,blanc))
    return noir != NbCode

In [8]:
def myplay():
    Code=[random.choice(myChoices) for _ in range(NbCode)]
    for attempt in range(NbMaxGuesses):
        if not running(Code):
            print('Win,',attempt+1,'tries')
            break
        else:
            print('Try',attempt+1)
    else:
        print('Game Over')

In [9]:
def initarbre():
    listepossible=[]
    for i1 in myChoices:
        for i2 in myChoices:
            for i3 in myChoices:
                for i4 in myChoices:
                    listepossible.append([i1,i2,i3,i4])
    return listepossible

In [2]:
NbCode=4
NbMaxGuesses=12
myChoices='roygbp'

In [22]:
class MM:
   
    def __init__(self,parent):
        self.parent = parent
        self.canvas=tk.Canvas(parent)
        self.status=tk.Label(parent)
        self.colors={'r':'red','o':'orange','y':'yellow','g':'green','b':'blue','p':'purple'}
        self.drawboard()
    
    def drawboard(self,event=None):
        self.canvas.destroy()
        self.status.destroy()
        self.canvas=tk.Canvas(self.parent,width=310, height=750)
        self.canvas.pack()
        self.bag={'r':self.canvas.create_oval(10,700,50,740,fill='red',outline='red'),
                  'o':self.canvas.create_oval(60,700,100,740,fill='orange',outline='orange'),
                  'y':self.canvas.create_oval(110,700,150,740,fill='yellow',outline='yellow'),
                  'g':self.canvas.create_oval(160,700,200,740,fill='green',outline='green'),
                  'b':self.canvas.create_oval(210,700,250,740,fill='blue',outline='blue'),
                  'p':self.canvas.create_oval(260,700,300,740,fill='purple',outline='purple')}
        self.ids={v:k for k,v in self.bag.items()}
        self.guesses=['']
        self.status=tk.Label(self.parent)
        self.canvas.create_line(0,680,400,680)
        self.canvas.create_line(0,60,400,60)
        self.canvas.create_text(150, 40,text="MASTERMIND", font=("Arial",30) )
        
        #empty circles
        for i in range(NbCode):
            for j in range(NbMaxGuesses):
                self.canvas.create_oval(10+i*50,630-j*50,50+i*50,670-j*50,outline='black')
        
        #empty squares
        for j in range(NbMaxGuesses):
            self.canvas.create_rectangle(230,640-50*j,255,650-50*j,outline='black')
            self.canvas.create_rectangle(260,640-50*j,285,650-50*j,outline='black')
            self.canvas.create_rectangle(230,655-50*j,255,665-50*j,outline='black')
            self.canvas.create_rectangle(260,655-50*j,285,665-50*j,outline='black')
        self.status.pack()
        
        #self.canvas.pack()
        #self.parent.bind('<Return>',exit)
        
        self.canvas.bind('<1>',self.check)
        self.parent.bind('<Control-n>',self.drawboard)
        self.parent.bind('<Control-N>',self.drawboard)
        self.Code=[random.choice(myChoices) for _ in range(NbCode)]
        self.CodeColorCount=collections.Counter(self.Code)
        print(self.Code)
    
    def check(self,event=None):
        if(len(self.canvas.find_withtag("current"))==0 
           or self.canvas.find_withtag("current")[0]> 6):
            return
        id=self.canvas.find_withtag("current")[0]
        guess=self.ids[id]
        self.guesses[-1]+=guess
        
        #print
        x_offset=10+(len(self.guesses[-1])-1)*50
        y_offset=630-(len(self.guesses)-1)*50
        self.canvas.create_oval(x_offset,y_offset,
                                x_offset+40,y_offset+40,
                                fill=self.colors[guess],outline='black')
        if len(self.guesses[-1])<4:
            return
        
        GuessColorCount=collections.Counter(self.guesses[-1])
        blanc=sum(min(self.CodeColorCount[k],GuessColorCount[k]) for k in self.CodeColorCount)
        noir=sum(a==b for a,b in zip(self.Code,self.guesses[-1]))
        blanc-=noir
        colors=noir*['black']+blanc*['yellow']
        key_coordinates=[(230,y_offset+10,255, y_offset+20),
                         (260,y_offset+10,285, y_offset+20),
                         (230,y_offset+25,255, y_offset+35),
                         (260,y_offset+25,285, y_offset+35)]
        for color,coord in zip(colors,key_coordinates):
            self.canvas.create_rectangle(coord,fill=color,outline='black')
        
        if noir==4:
            self.status.config(text='You win in {} guesses.'.format(len(self.guesses)))
            self.canvas.unbind('<1>')
        elif len(self.guesses) > NbMaxGuesses-1:
            self.status.config(text='Game over! The correct answer is {}.'.format(''.join(self.Code)))
            self.canvas.unbind('<1>')
        else:
            self.guesses.append('')

In [23]:
root=tk.Tk()
game=MM(root)
root.mainloop()

['o', 'g', 'r', 'g']


In [54]:
#init
def play(show=True,imposedcode=False,Code=[]):
    if imposedcode==False:
        Code=[random.choice(myChoices) for _ in range(NbCode)]
    
    print(Code)
    
    listepossible=initarbre()

    #storage
    guesslist=[]
    scorelist=[]

    #first guess
    guesslist.append([random.choice(myChoices) for _ in range(NbCode)])
    scorelist.append(score(Code,guesslist[0]))

    #evaluation
    for attempt in range(NbMaxGuesses):
        if scorelist[attempt][0]==4:
            break
        else:
            newlist=[]
            for idx,i in enumerate(listepossible):
                if score(guesslist[attempt],i)==scorelist[attempt]:
                    newlist.append(i)
            if show:
                print(attempt+1,'->',*guesslist[attempt],scorelist[attempt])
                print('Search in :',len(newlist))

            guesslist.append(newlist[0])
            scorelist.append(score(Code,guesslist[attempt+1]))
            listepossible = deepcopy(newlist)

    else:
        if show:
            print('Game Over')
    if show:
        print('\n')
        print("Computer wins in",attempt+1,"attempts")
        print('Code',*guesslist[attempt])

    return attempt+1

In [55]:
#MULTI=1000
#multiplaylist=[]
#for _ in range(MULTI):
#    multiplaylist.append(play(show=False,imposedcode=False))
    
#print('moyenne de',MULTI,"jeux:",np.mean(multiplaylist))

In [12]:
mychoix=input('Qui trouve la solution? moi ou toi?')

if mychoix=='moi':
    myplay()
else:
    mychoix2=input('Tu veux renseigner un code? (oui/non)')
    if mychoix2=='oui':
        Code=[]
        mycode=input('Code:')
        for i in range(NbCode):
            Code.append(mycode[i])
        myplay(show=True,imposedcode=True,Code=Code)
    else:
        myplay(show=True,imposedcode=False)

Qui trouve la solution? moi ou toi?toi
Tu veux renseigner un code? (oui/non)non


TypeError: myplay() got an unexpected keyword argument 'show'